### CNN

In [25]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import KFold

#### Load Data

In [2]:
from src.dataLoading import dataLoader
from sklearn.model_selection import train_test_split
    
vectors, images, labels = dataLoader()  # 784-long vectors, 28*28 images and mnist/chinese labels

encoder = LabelBinarizer()
transfomed_labels = encoder.fit(np.unique(labels))
# split the vectors (for PCA, CNN would use images)
# "stratify" makes sure theres a balance of each class in the test/train sets
X_train, X_test, y_train, y_test = train_test_split(vectors, labels, train_size=0.8, stratify=labels)
y_train_t = encoder.transform(y_train)
y_test_t = encoder.transform(y_test)

In [19]:
X_train = np.reshape(X_train, (X_train.shape[0],28,28,1))
X_test = np.reshape(X_test, (X_test.shape[0],28,28,1))

In [4]:
X_train.shape

(16000, 28, 28, 1)

In [5]:
y_train_t.shape

(16000, 20)

In [7]:
X_test.shape

(4000, 28, 28, 1)

In [8]:
y_test_t.shape

(4000, 20)

In [22]:
def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(20))
    return model

In [28]:
model = create_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)               

In [29]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, y_train_t, epochs=10,
                    validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 5s 10ms/step - loss: 0.8769 - accuracy: 0.8113 - val_loss: 0.1865 - val_accuracy: 0.9367
Epoch 2/10
422/422 [==============================] - 4s 9ms/step - loss: 0.1387 - accuracy: 0.9612 - val_loss: 0.1162 - val_accuracy: 0.9640
Epoch 3/10
422/422 [==============================] - 4s 9ms/step - loss: 0.0829 - accuracy: 0.9761 - val_loss: 0.0982 - val_accuracy: 0.9707
Epoch 4/10
422/422 [==============================] - 4s 8ms/step - loss: 0.0690 - accuracy: 0.9799 - val_loss: 0.0715 - val_accuracy: 0.9747
Epoch 5/10
422/422 [==============================] - 4s 9ms/step - loss: 0.0505 - accuracy: 0.9853 - val_loss: 0.1032 - val_accuracy: 0.9693
Epoch 6/10
422/422 [==============================] - 4s 9ms/step - loss: 0.0423 - accuracy: 0.9881 - val_loss: 0.1902 - val_accuracy: 0.9473
Epoch 7/10
422/422 [==============================] - 4s 11ms/step - loss: 0.0533 - accuracy: 0.9856 - val_loss: 0.1245 - val_accuracy: 0.9700
Epoc

In [31]:
acc_per_fold = []
loss_per_fold = []
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_n = 1
for train, test in kfold.split(X_train, y_train_t):
    print(f'FOLD {fold_n}')
    model = create_model()
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    history = model.fit(
        X_train[train],
        y_train_t[train],
        epochs=10,
        # batch_size=64,
        validation_split=0.1,
        verbose=2,
        # callbacks=callbacks
    )
    test_loss, test_acc = model.evaluate(X_train[test], y_train_t[test], verbose=2)
    print(f'Score for fold {fold_n}: {model.metrics_names[0]} of {test_loss}; {model.metrics_names[1]} of {test_acc*100}%')
    acc_per_fold.append(test_acc * 100)
    loss_per_fold.append(test_loss)
    fold_n += 1

FOLD 1
Epoch 1/10
380/380 - 4s - loss: 0.8665 - accuracy: 0.8060 - val_loss: 0.1679 - val_accuracy: 0.9548
Epoch 2/10
380/380 - 3s - loss: 0.1378 - accuracy: 0.9609 - val_loss: 0.1315 - val_accuracy: 0.9637
Epoch 3/10
380/380 - 3s - loss: 0.0882 - accuracy: 0.9753 - val_loss: 0.0971 - val_accuracy: 0.9763
Epoch 4/10
380/380 - 3s - loss: 0.0552 - accuracy: 0.9830 - val_loss: 0.0904 - val_accuracy: 0.9815
Epoch 5/10
380/380 - 3s - loss: 0.0517 - accuracy: 0.9840 - val_loss: 0.1176 - val_accuracy: 0.9652
Epoch 6/10
380/380 - 3s - loss: 0.0444 - accuracy: 0.9863 - val_loss: 0.1413 - val_accuracy: 0.9667
Epoch 7/10
380/380 - 3s - loss: 0.0358 - accuracy: 0.9899 - val_loss: 0.1114 - val_accuracy: 0.9726
Epoch 8/10
380/380 - 3s - loss: 0.0454 - accuracy: 0.9858 - val_loss: 0.1059 - val_accuracy: 0.9800
Epoch 9/10
380/380 - 3s - loss: 0.0393 - accuracy: 0.9892 - val_loss: 0.0938 - val_accuracy: 0.9763
Epoch 10/10
380/380 - 3s - loss: 0.0325 - accuracy: 0.9910 - val_loss: 0.1307 - val_accuracy:

In [ ]:
np.mean(acc_per_fold)